# Working with Census Data for Predicting whether the Yearly Income Exceeds 50K 

# Objective: 
To predict whether income exceeds 50K/yr based on census data

Dataset: Adult Data Set

https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data


Variable description:
    
age: continuous

workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

fnlwgt: continuous.

education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

education-num: continuous.

marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

sex: Female, Male.

capital-gain: continuous.

capital-loss: continuous.

hours-per-week: continuous.

native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

class: >50K, <=50K

## Import Libraries

In [ ]:
# Pandas and Numpy libraries
import pandas as pd
import numpy as np

In [ ]:
# For preprocessing the data
from sklearn.preprocessing import Imputer
from sklearn import preprocessing

In [ ]:
# To split the dataset into train and test datasets
from sklearn.model_selection import train_test_split

In [ ]:
# To model the Gaussian Navie Bayes classifier
from sklearn.naive_bayes import GaussianNB

In [ ]:
# To calculate the accuracy score of the model
from sklearn.metrics import accuracy_score

## Read Data

In [ ]:
adult_df = pd.read_csv('adult.data', header = None, delimiter=' *, *', engine='python')

Load the dataset. Observe that this file has .data extention

For importing the census data, we are using pandas read_csv() method. This method is a very simple and fast method for importing 
data.

We are passing four parameters. The ‘adult.data’ parameter is the file name. The header parameter is for giving details to pandas
that whether the first row of data consists of headers or not. In our dataset, there is no header. So, we are passing None.

The delimiter parameter is for giving the information the delimiter that is separating the data. Here, we are using ‘ , ’ 
delimiter. This delimiter is to show delete the spaces before and after the data values. This is very helpful when there is 
inconsistency in spaces used with data values.

In [ ]:
# Print columns in the adult data set
adult_df.columns

In [ ]:
# Adding headers to the dataframe 
adult_df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
                    'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

In [ ]:
# Number of records(rows) in the dataframe
len(adult_df)

## Handling Missing Data

In [ ]:
# Test whether there is any null value in our dataset or not. We can do this using isnull() method.
adult_df.isnull().sum()

The above output shows that there is no “null” value in our dataset.

Let’s try to test whether any categorical attribute contains a “?” in it or not. 

At times there exists “?” or ” ” in place of missing values. 
Using the below code snippet we are going to test whether adult_df data frame consists of categorical variables with values as “?”.

In [ ]:
for value in ['workclass','education','marital_status','occupation','relationship','race','sex','native_country','income']:
    print(value,":", sum(adult_df[value] == '?'))

The output of the above code snippet shows that there are 1836 missing values in workclass attribute. 1843 missing values in 
occupation attribute and 583 values in native_country attribute.

## Data Preprocessing

For preprocessing, we are going to make a duplicate copy of our original dataframe.

We are duplicating adult_df to adult_df_rev dataframe. 

Observe that we have used deep copy while copying. Why?

In [ ]:
## Deep copy of adult_df
adult_df_rev = adult_df.copy(deep=True)

Before doing missing values handling task, we need some summary statistics of our dataframe. For this, we can use describe() 
method. It can be used to generate various summary statistics, excluding NaN values.

In [ ]:
adult_df_rev.describe()

We are passing an “include” parameter with value as “all”, this is used to specify that. 

We want summary statistics of all the attributes.

In [ ]:
adult_df_rev.describe(include= 'all')

## Data Imputation 

Some of the categorical values have missing values i.e, “?”. 

We replace the “?” with the above describe methods top row’s value. 

For example, we replace the “?” values of workplace attribute with “Private” value.

In [ ]:
for value in ['workclass','education','marital_status','occupation','relationship','race','sex','native_country','income']:
    #adult_df_rev[value].replace(['?'], [adult_df_rev.describe(include='all')[value][2]],inplace='True')
    replaceValue = adult_df_rev.describe(include='all')[value][2]
    adult_df_rev[value][adult_df_rev[value]=='?'] = replaceValue

In [ ]:
print(adult_df.workclass)

## Integer Encoding to Categorical Data 

For Naive Bayes, we need to convert all the data values in one format.

We are going to encode all the labels with the value between 0 and n_classes-1. 

For implementing this, we are going to use LabelEncoder of scikit learn library.

In [ ]:
le = preprocessing.LabelEncoder()

workclass_cat = le.fit_transform(adult_df.workclass)
education_cat = le.fit_transform(adult_df.education)
marital_cat   = le.fit_transform(adult_df.marital_status)
occupation_cat = le.fit_transform(adult_df.occupation)
relationship_cat = le.fit_transform(adult_df.relationship)
race_cat = le.fit_transform(adult_df.race)
sex_cat = le.fit_transform(adult_df.sex)
native_country_cat = le.fit_transform(adult_df.native_country)

In [ ]:
print(workclass_cat)

## Create New Columns with Integer-Encoded Categories for Each Feature

In [ ]:
# Create and Initialize the encoded categorical columns

adult_df_rev['workclass_cat'] = workclass_cat
adult_df_rev['education_cat'] = education_cat
adult_df_rev['marital_cat'] = marital_cat
adult_df_rev['occupation_cat'] = occupation_cat
adult_df_rev['relationship_cat'] = relationship_cat
adult_df_rev['race_cat'] = race_cat
adult_df_rev['sex_cat'] = sex_cat
adult_df_rev['native_country_cat'] = native_country_cat

In [ ]:
adult_df_rev.head()

## Discard the Original Categorical Columns

In [ ]:
# Drop the old/original categorical columns from dataframe
dummy_fields = ['workclass','education','marital_status','occupation','relationship','race', 'sex', 'native_country']
adult_df_rev = adult_df_rev.drop(dummy_fields, axis = 1)

Reindex all the columns properly. We have passed the list of column names as a parameter and axis=1 for reindexing the columns.

In [ ]:
adult_df_rev = adult_df_rev.reindex_axis(['age', 'workclass_cat', 'fnlwgt', 'education_cat',
                                    'education_num', 'marital_cat', 'occupation_cat',
                                    'relationship_cat', 'race_cat', 'sex_cat', 'capital_gain',
                                    'capital_loss', 'hours_per_week', 'native_country_cat', 
                                    'income'], axis= 1)
adult_df_rev.head(5)

Now we have created multiple categorical columns like “marital_cat”, “race_cat” etc. !

# Data Slicing

In [ ]:
# Arrange data into independent variables and dependent variables
X = adult_df_rev.values[:,:14]  ## Features
Y = adult_df_rev.values[:,14]  ## Target...14th column is the Target column

In [ ]:
# Split the data into train and test
# Train data size: 70% of original data
# Test data size: 30% of original data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3, random_state = 10)

## Build the Naive Bayes Model

Implement Gaussian Naive Bayes

In [ ]:
# Build the object
clf = GaussianNB()

# Train the model with Training data
clf.fit(X_train, Y_train)

Now **GaussianNB** classifier is built. 

The classifier is trained using training data. 

We can use **fit()** method for training it. 

After building a classifier, our model is ready to make predictions. 

We can use **predict()** method with test set features as its parameters.

## Evaluate the Model on Test Data

In [ ]:
Y_pred = clf.predict(X_test)

## Model Accuracy
Accuracy of our Gaussian Naive Bayes model

In [ ]:
accuracy_score(Y_test, Y_pred, normalize = True)